## Prepare

In [ ]:
# Import libraries and load Street View API key

import streetview
import geopandas as gpd
import pandas as pd
import shapely

import json
with open("config.json", "r") as f:
    config = json.load(f)
api_key = config["street_view_api_key"]["key"]

In [ ]:
# Read in dataframe of needed coordinates from previous script
coordinates_df = gpd.read_file('data/street_view_coordinates.geojson')

## Get images from Street View API

In [ ]:
pano_data = []

for index, row in coordinates_df.iterrows():
        
    # Properties of coordinates to be retrieved
    point_lon = row['lon']
    point_lat = row['lat']
    point_heading = row['direction']
    point_file = row['filename']
    
    try:
        # Get Street View metadata at given point
        panos_raw = pd.DataFrame(streetview.search_panoramas(lat=point_lat, lon=point_lon))
        
        panos_clean = pd.DataFrame([])
        panos_clean['pano_id'] = [element[1] for element in panos_raw[0]]
        panos_clean['pano_lon'] = [element[1] for element in panos_raw[2]]
        panos_clean['pano_lat'] = [element[1] for element in panos_raw[1]]
        panos_clean['pano_date'] = [element[1] for element in panos_raw[6]]
        
        # Select most recent Street View image available
        pano_selected = panos_clean.sort_values(by='pano_date', ascending=False).head(1)
        pano_selected_id = pano_selected['pano_id'].iloc[0]
        
        # Get image
        image = streetview.get_streetview(
            pano_id=pano_selected_id,
            api_key=api_key,
            heading=point_heading
        )

        # Save image using predefined name
        image.save("street_view_images/" + point_file, "jpeg")
       
    # API will sometimes return out-of-index errors; not too many instances, so can skip 
    except:
        pano_selected = pd.DataFrame({'pano_id': None, 'pano_lon': None, 'pano_lat': None, 'pano_date': None}, index=[0])
        print(f"Error on row index {index}, continuing")
        
    pano_data.append(pano_selected)

Error on row index 238, continuing
Error on row index 239, continuing
Error on row index 240, continuing
Error on row index 241, continuing
Error on row index 248, continuing
Error on row index 249, continuing


## Finish up

In [ ]:
# Join Street View metadata with prepared coordinates metadata, then write out combined dataframe
pano_data_collated = pd.concat(pano_data).reset_index(drop=True)

data_out = pd.concat([coordinates_df, pano_data_collated], axis=1)

data_out.to_file("data/street_view_coordinates_with_metadata.geojson", driver='GeoJSON')

/var/folders/_9/_8f0yv7d1vn6dkv6325fg4gm0000gq/T/ipykernel_33402/1813084926.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pano_data_collated = pd.concat(pano_data).reset_index(drop=True)


In [33]:
# Testing that the coordinates from retrieved Street View images are sensible

street_view_points = data_out
street_view_points['streetview_geometry'] = gpd.points_from_xy(data_out['pano_lon'], data_out['pano_lat'])
street_view_points.set_geometry('streetview_geometry')

street_view_points.explore(tiles = "CartoDB positron")